In [ ]:
#import zipfile
import numpy as np
import pandas
import os
import io
from pprint import pprint
import json

In [ ]:
# import list of teams from retrosheet
teams_df = pandas.DataFrame.from_csv('teams.csv')
teams_df = teams_df.fillna('')
teams_df = teams_df.drop_duplicates(subset='teamID')

teamarray = teams_df[['teamID', 'league']].values.tolist()

#teams_df = teams_df.drop_duplicates(subset='teamID')

In [ ]:
#os.chdir('retrosheet/')
#os.getcwd()

#decades = np.linspace(1920,2010,10)
#zipfiles = []
#for x in decades:
#    zipfiles.append('{:.0f}'.format(x)+'seve.zip')
#for filename in zipfiles:
#    zip_ref = zipfile.ZipFile(filename, 'r')
#    zip_ref.extractall('.')
#    zip_ref.close()

#os.chdir('../')
#print('Done')
os.getcwd()

In [ ]:
allfiles = []

yeararray = np.linspace(1921,2016,96)
#yeararray = [1927,1938]
#yeararray = np.linspace(2012,2016,5)

for oneyear in yeararray:
    for oneteam in teamarray:
        oneteamname = oneteam[0]
        itsleague = oneteam[1][0:1]
        afile = 'retrosheet/' + '{:.0f}'.format(oneyear)+oneteamname+'.EV'+itsleague
        if (os.path.isfile(afile)):
            allfiles.append(afile)

#print(allfiles)
print('files identified')

In [ ]:
allgames = []
gamecount = 0
#allfiles=['retrosheet/1995MON.EVN']
for filename in allfiles:
    print(filename)
    for line in io.open(filename,encoding='utf-8').readlines():
        linearray = line.split(',')
#        print(linearray)
        linearray[-1] = linearray[-1][0:-1]  # cut off newline character
        if (linearray[0] == 'id'):
            # 'id' starts the next game, so immediately advance gamecount  
            # record prior game data by adding it to the gamelogs list, unless gamecount = -1
            if (gamecount > 0):
                thisgamedict['info'] = info_dict
                thisgamedict['visiting_starters'] = vis_start_list.tolist()
                thisgamedict['home_starters'] = home_start_list.tolist()
                thisgamedict['events'] = events.tolist()
                thisgamedict['data'] = data_list.tolist()
                # append the dictionary describing this game to the list of games
                allgames.append(thisgamedict)

            # advance the gamecount and initialize a new game
            gamecount = gamecount + 1
            thisgamedict = {}
            info_dict = {}
            start_dict = {}
            vis_start_list = np.empty((0,4),dtype=object)
            home_start_list = np.empty((0,4),dtype=object)
            vis_sub_list = np.empty((0,4),dtype=object)
            home_sub_list = np.empty((0,4),dtype=object)
            events = np.empty((0,12),dtype=object)
            data_list = np.empty((0,2),dtype=object)
            eventnum = 1

            # the first thing to go into the new game is the game id.
            # the [0:-1] is there to cut off the newline character
            thisgamedict['id'] = linearray[1]

        elif (linearray[0] == 'version'):
            thisgamedict['version'] = linearray[1]
        elif (linearray[0] == 'info'):
            info_dict[linearray[1]] = linearray[2]
        elif (linearray[0] == 'start'):
            thisbatter = np.empty((1,4),dtype=object)
            if (linearray[3] == '0'):
                thisbatter[0,0] = linearray[1]
                thisbatter[0,1] = linearray[2]
                thisbatter[0,2] = linearray[4]
                thisbatter[0,3] = linearray[5]
                vis_start_list = np.vstack((vis_start_list, thisbatter))
            elif (linearray[3] == '1'):
                thisbatter[0,0] = linearray[1]
                thisbatter[0,1] = linearray[2]
                thisbatter[0,2] = linearray[4]
                thisbatter[0,3] = linearray[5]
                home_start_list = np.vstack((home_start_list, thisbatter))
            else:
                print('NEITHER VISITOR NOR HOME????')
                
        elif (linearray[0] == 'data'):
            thisdata = np.empty((2),dtype=object)
            thisdata[0] = linearray[2]
            thisdata[1] = linearray[3]
            data_list = np.vstack((data_list,thisdata))
        else:
#            print(linearray)
            thisevent = []
            thisevent.append(eventnum)
            thisevent.append(linearray[0])  # say what type of event: play/com/sub/etc.
            if (linearray[0] == 'play'):
                for i in range(1,7):
                    thisevent.append(linearray[i])
                for i in range(8,12):   # comment, sub_batting_order, sub_position empty
                    thisevent.append('')
            elif (linearray[0] == 'com'):
                for i in range(1,7):
                    thisevent.append('')
                thisevent.append(linearray[1])
                for i in range(9,12):
                    thisevent.append('')
            elif (linearray[0] == 'sub'):
                thisevent.append('')    # inning, fill in later
                thisevent.append('')    # batting team (vis/home), fill in later
                thisevent.append(linearray[1])  # playerID of player coming in
                for i in range(4,8):
                    thisevent.append('')  # count/pitches/result/comment all blank
                thisevent.append(linearray[3]) # batting or fielding sub
                thisevent.append(linearray[4]) # sub's batting order
                thisevent.append(linearray[5]) # sub's fielding position
            elif (linearray[0] == 'badj'):
                thisevent.append('')    # inning, fill in later
                thisevent.append('')  # visitor/home, fill in later
                for i in range(4,8):
                    thisevent.append('')  # count/pitches/result/comment all blank
                thisevent.append('')    # batting/fielding - fill in as batting team later
                thisevent.append(linearray[1])  # playerID of adjusting batter
                thisevent.append(linearray[2]) # batter hand adjusted to
                thisevent.append('')  # add blank                
            elif (linearray[0] == 'padj'):
                thisevent.append('')    # inning, fill in later
                thisevent.append('')  # visitor/home, fill in later
                for i in range(4,8):
                    thisevent.append('')  # count/pitches/result/comment all blank                
                thisevent.append(linearray[1])  # playerID of adjusting pitcher
                thisevent.append('')    # batting/fielding - fill in as fielding team later
                thisevent.append('')  # add blank                                
                thisevent.append(linearray[2]) # pitcher hand adjusted to
            elif (linearray[0] == 'ladj'):
                thisevent.append('')    # inning, fill in later
                thisevent.append(linearray[1])  # visitor/home
                for i in range(3,8):
                    thisevent.append('')  # playerID/count/pitches/result/comment all blank
                thisevent.append('')    # batting/fielding - fill in as fielding team later
                thisevent.append(linearray[2]) # new batter
                thisevent.append('') # add blank  
            else:
                print(linearray)
            events = np.vstack((events,thisevent))
            eventnum = eventnum + 1

#for i in range(0,len(allgames)):
#    print('i = ',i)
#    pprint(allgames[i])
#    print('\n')
print('ahoy')
#pprint(allgames)
with open('events_raw.json', 'w') as f:
    json.dump(allgames, f)
print('ok')

In [ ]:
pprint(allgames[-1]['id'])
print(len(allgames))